# # Film Eleştirileri ve Bag-of-Words Modellemesi

🎯 Bu zorluğun amacı, metinlerin ***Bag-of-words*** modellemesiyle oynamaktır.

✍️ Aşağıdaki veri setinde, _“olumlu”_ veya _“olumsuz”_ olarak sınıflandırılmış 2000 adet yorum bulunmaktadır.

In [1]:
import pandas as pd

data = pd.read_csv("https://d32aokrjazspmn.cloudfront.net/materials/movie_reviews.csv")
data.head()

,target,reviews
0,neg,"plot : two teen couples go to a church party ,..."
1,neg,the happy bastard's quick movie review \ndamn ...
2,neg,it is movies like these that make a jaded movi...
3,neg,""" quest for camelot "" is warner bros . ' firs..."
4,neg,synopsis : a mentally unstable man undergoing ...


In [2]:
data.shape

(2000, 2)

In [3]:
import pandas as pd
import string
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_validate

# Gerekli NLTK paketlerini indirdiğimizden emin olalım (önceki ödevden kalmış olabilir ama garanti olsun)
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

# Veriyi yüklüyoruz
data = pd.read_csv("https://d32aokrjazspmn.cloudfront.net/materials/movie_reviews.csv")

# Verinin boyutuna ve ilk 5 satırına bakalım
print("Veri Boyutu:", data.shape)
data.head()

[nltk_data] Downloading package punkt to /home/tumay/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/tumay/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/tumay/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Veri Boyutu: (2000, 2)


,target,reviews
0,neg,"plot : two teen couples go to a church party ,..."
1,neg,the happy bastard's quick movie review \ndamn ...
2,neg,it is movies like these that make a jaded movi...
3,neg,""" quest for camelot "" is warner bros . ' firs..."
4,neg,synopsis : a mentally unstable man undergoing ...


## 1. Ön işleme (Preprocessing)

❓ **Soru (Metin Temizleme)** ❓

- Bir cümleyi temizleyecek bir `preprocessing` fonksiyonu yazın ve bunu tüm yorumlara uygulayın. Fonksiyon şunları yapmalıdır:
    - boşlukları kaldırma
    - harfleri küçük harfe çevirme
    - sayıları kaldırma
    - noktalama işaretlerini kaldırma
    - tokenization (kelimelere ayırma)
    - lemmatization (kelime köküne indirgeme)
- Temizlenmiş yorumları `clean_reviews` adlı bir sütunda saklayabilirsiniz.
- Bu aşamada stopword’leri kaldırmayın; nedenini `3. N-gram modelleme` bölümünde açıklayacağız.

In [5]:
import string
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

def preprocessing(sentence):
    # 1. Boşlukları (whitespace) temizle
    sentence = sentence.strip()
    
    # 2. Küçük harfe çevir
    sentence = sentence.lower()
    
    # 3. Sayıları temizle
    sentence = "".join([char for char in sentence if not char.isdigit()])
    
    # 4. Noktalama işaretlerini temizle
    # string.punctuation içindeki semboller hariç her şeyi al
    sentence = "".join([char for char in sentence if char not in string.punctuation])
    
    # 5. Tokenization (Kelimelere ayır)
    tokens = word_tokenize(sentence)
    
    # 6. Lemmatization (Kök bulma)
    lemmatizer = WordNetLemmatizer()
    # Fiil (verb) olarak kök bulmak bazen daha iyi sonuç verir ama standart kullanım (isim) yeterli şimdilik.
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # 7. Tekrar birleştir (Join)
    return " ".join(lemmatized_tokens)

# Fonksiyonu tüm yorumlara uygulama kısmı:
# data['review'] sütununu al, temizle ve data['clean_reviews'] sütununa yaz.
data['clean_reviews'] = data['reviews'].apply(preprocessing)

# Kontrol edelim
data.head()


,target,reviews,clean_reviews
0,neg,"plot : two teen couples go to a church party ,...",plot two teen couple go to a church party drin...
1,neg,the happy bastard's quick movie review \ndamn ...,the happy bastard quick movie review damn that...
2,neg,it is movies like these that make a jaded movi...,it is movie like these that make a jaded movie...
3,neg,""" quest for camelot "" is warner bros . ' firs...",quest for camelot is warner bros first feature...
4,neg,synopsis : a mentally unstable man undergoing ...,synopsis a mentally unstable man undergoing ps...


In [ ]:
# Yorumları temizle
pass  # SENİN KODUN BURAYA

❓ **Soru (LabelEncoding)**❓

Hedefinizi LabelEncode ile kodlayın ve `“target_encoded”` adlı bir sütuna kaydedin.

In [6]:
from sklearn.preprocessing import LabelEncoder

# 1. Kodlayıcıyı (Encoder) çağırıyoruz
encoder = LabelEncoder()

# 2. Hedef sütunu (target) sayıya çeviriyoruz
# DİKKAT: Veri setindeki hedef sütununun adı 'target' olmayabilir.
# 'polarity', 'sentiment' veya 'label' olabilir. data.columns çıktısına bakıp doğru ismi yazmalısın.
# Ben varsayılan olarak 'target' yazdım.
data['target_encoded'] = encoder.fit_transform(data['target'])

# 3. Kontrol
data.head()# SENİN KODUN BURAYA

,target,reviews,clean_reviews,target_encoded
0,neg,"plot : two teen couples go to a church party ,...",plot two teen couple go to a church party drin...,0
1,neg,the happy bastard's quick movie review \ndamn ...,the happy bastard quick movie review damn that...,0
2,neg,it is movies like these that make a jaded movi...,it is movie like these that make a jaded movie...,0
3,neg,""" quest for camelot "" is warner bros . ' firs...",quest for camelot is warner bros first feature...,0
4,neg,synopsis : a mentally unstable man undergoing ...,synopsis a mentally unstable man undergoing ps...,0


In [7]:
# Hızlı kontrol
data

,target,reviews,clean_reviews,target_encoded
0,neg,"plot : two teen couples go to a church party ,...",plot two teen couple go to a church party drin...,0
1,neg,the happy bastard's quick movie review \ndamn ...,the happy bastard quick movie review damn that...,0
2,neg,it is movies like these that make a jaded movi...,it is movie like these that make a jaded movie...,0
3,neg,""" quest for camelot "" is warner bros . ' firs...",quest for camelot is warner bros first feature...,0
4,neg,synopsis : a mentally unstable man undergoing ...,synopsis a mentally unstable man undergoing ps...,0
...,...,...,...,...
1995,pos,wow ! what a movie . \nit's everything a movie...,wow what a movie it everything a movie can be ...,1
1996,pos,"richard gere can be a commanding actor , but h...",richard gere can be a commanding actor but he ...,1
1997,pos,"glory--starring matthew broderick , denzel was...",glorystarring matthew broderick denzel washing...,1
1998,pos,steven spielberg's second epic film on world w...,steven spielberg second epic film on world war...,1


In [8]:
# Hangi sınıfın hangi numarayı aldığını görmek için:
print(encoder.classes_)

['neg' 'pos']


## 2. Bag-of-Words Modellemesi

❓ **Soru (Tek kelimelik sözcüklerle NaiveBayes)** ❓

`cross_validate` kullanarak, metinlerin Bag-of-Words temsilinde eğitilmiş bir Multinomial Naive Bayes modelini puanlayın.

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_validate

# 1. Vektörleştiriciyi Kur (Sözlük Oluşturucu)
vectorizer = CountVectorizer()

# 2. Metinleri Sayıya Çevir (Matris Oluştur)
# data['clean_reviews'] sütununu alıp sayısal haritaya dönüştürüyoruz.
X_bow = vectorizer.fit_transform(data['clean_reviews'])

# 3. Modeli Tanımla
nb_model = MultinomialNB()

# 4. Çapraz Doğrulama (Sınav Yap)
# cv_results değişkenine sınav sonuçlarını kaydediyoruz.
# scoring='accuracy' diyerek "Doğruluk" oranını ölçmesini istiyoruz.
cv_results = cross_validate(nb_model, X_bow, data['target_encoded'], cv=5, scoring='accuracy')

# 5. Sonucu Yazdır
# test_score'ların ortalamasını alıyoruz.
print("Ortalama Doğruluk (Accuracy):", cv_results['test_score'].mean())

Ortalama Doğruluk (Accuracy): 0.8165000000000001


## 3. N-gram Modellemesi

👀 Stop kelimeleri kaldırmamanızı istediğimizi hatırlayın. Neden? 

👉 Naive Bayes modelini bigramlarla eğiteceğiz. Bu nedenle, “I do not like coriander” (Kişnişi sevmiyorum) gibi bir cümlede, örneğin bu cümlede olumsuzluğu tespit etmek için “do not” bigramını taramak önemlidir.

❓ **Soru (bigramlarla NaiveBayes)** ❓

`cross_validate` kullanarak, metinlerin 2-gram Bag-of-Words temsilinde eğitilmiş bir Multinomial Naive Bayes modelini puanlayın.

In [10]:
vectorizer = CountVectorizer(ngram_range = (2,2))
naivebayes = MultinomialNB()

X_bow = vectorizer.fit_transform(data.clean_reviews)

cv_nb = cross_validate(
    naivebayes,
    X_bow,
    data.target_encoded,
    scoring = "accuracy"
)

round(cv_nb['test_score'].mean(),2)

np.float64(0.84)

In [11]:
# 1. Vektörleştiriciyi (Bigram Modunda) Ayarla
# ngram_range=(2, 2) demek: "En az 2, en çok 2 kelimelik gruplar oluştur."
# Yani sadece ikilileri al. (Eğer (1, 2) deseydik hem teklileri hem ikilileri alırdı).
vectorizer = CountVectorizer(ngram_range=(2, 2))

# 2. Metinleri Sayıya Çevir (Bigram Matrisi)
X_bow = vectorizer.fit_transform(data['clean_reviews'])

# 3. Modeli Tanımla
nb_model = MultinomialNB()

# 4. Çapraz Doğrulama (Sınav Yap)
cv_results = cross_validate(nb_model, X_bow, data['target_encoded'], cv=5, scoring='accuracy')

# 5. Sonucu Yazdır
print("Bigram Modeli Doğruluk Skoru:", cv_results['test_score'].mean())

Bigram Modeli Doğruluk Skoru: 0.8400000000000001


İlk model "Kelime Avcısı"ydı, ikinci model ise "Kalıp Avcısı" oldu. O %3'lük artış, modelin bağlamı (context) anlamaya başladığının ispatıdır.

🏁 Tebrikler! Artık vektörleştirilmiş metinler üzerinde Naive Bayes modelini nasıl eğiteceğinizi biliyorsunuz.

💾 Not defterinizi `git add/commit/push` yapmayı unutmayın...

